In [2]:
import os
import time
import csv
import re
import yaml
import inspect
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

ModuleNotFoundError: No module named 'selenium'

In [ ]:
# Get the current working directory
current_dir = os.getcwd()

# Move up a couple of folders
project_dir = os.path.abspath(os.path.join(current_dir, "../../"))

In [ ]:
# Define the path to your YAML file
yaml_file = os.path.abspath(os.path.join(current_dir, "../../config.yaml"))

# Read the YAML file
with open(yaml_file, 'r') as file:
    config = yaml.safe_load(file)

# Access the values from the YAML file
vessel_type = config['vessel_type']
class_type = config['class']
base = config['base']
quota = str(config['quota'])

In [ ]:
#edit sesuai kebutuhan
num = base + 1
filenum = str(num)

In [ ]:
# Directory file csv
read_path = os.path.join(project_dir,"input/class",class_type,vessel_type,quota,"split_{}.csv".format(filenum))
write_path = os.path.join(project_dir,"output",class_type, vessel_type,"{}.csv".format(filenum))

In [ ]:
print(read_path)
print(write_path)

In [ ]:
# Check if input file exists
if os.path.exists(read_path):
    print(f"CSV read file is found, continuing...")
    
else:
    print(f"CSV read file is not found")
    print('exiting....')
    
# Check if output file exists and create it if necessary
if os.path.exists(write_path):
    print(f"CSV write file  is found, continuing...")
else:
    print(f"CSV write file is not found, creating one...")
    os.makedirs(os.path.dirname(write_path), exist_ok=True) # Create output directory if it doesn't exist
    with open(write_path, 'w', newline='') as f:
        writer = csv.writer(f)

In [ ]:
def search(IMO):
    driver.get("https://marine-offshore.bureauveritas.com/bv-fleet/#/bv-fleet/")
    actions = ActionChains(driver)
    
    try:
        # Cookie button
        cookie_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/div[3]/div[1]/button[1]')))
        driver.execute_script("arguments[0].click();", cookie_button)
    except Exception as e:
        pass
    
    # Search bar
    search_field = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/main/div/div/div[2]/div/app-root/div/app-ship-basic-search/div/div/form/fieldset/div[1]/div/div/div[1]/input')))
    search_field.send_keys(IMO)
    
    # Search button
    search_button = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/main/div/div/div[2]/div/app-root/div/app-ship-basic-search/div/div/form/fieldset/div[2]/div[1]/div/button')))
    driver.execute_script("arguments[0].click();", search_button)

    # Ships button
    ship_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/main/div/div/div[2]/div/app-root/div/app-ship-basic-search/div/div/div[2]/ul/li/div/div[1]/a[1]')))
    driver.execute_script("arguments[0].click();", ship_button)



In [ ]:
def extract_numeric_value(text):
    # Use regex to remove letters and keep digits and specific non-numeric characters
    cleaned_text = re.sub(r'[^0-9()\[\]{}.,;:<>]', '', text)
    
    # Use regex to extract the numeric value from the cleaned text
    match = re.search(r'\d+\.?\d*', cleaned_text)
    if match:
        return match.group()
    else:
        return None

In [ ]:
def find_xpath_by_text1(driver, target_text):
    base_row_xpath = "/html/body/app-popup/div/div[2]/div[2]/app-ship-details/div/div/table[1]/tbody/tr[NUMBER]/td[1]"
    
    for number in range(2, 42, 2):  # Iterate from 2 to 40 (inclusive), with a step of 2
        row_xpath = base_row_xpath.replace("NUMBER", str(number))
        
        # Use WebDriverWait to wait for the element to be present and visible
        try:
            element = WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.XPATH, str(row_xpath))))
            
            if target_text in element.text:
                # If the target_text is found in the row, get the corresponding content xpath
                content_xpath = row_xpath.replace("/td[1]", "/td[2]")
                return content_xpath
        except:
            pass

    # Return None if the target_text is not found after 20 attempts
    return None


In [ ]:
def find_xpath_by_text2(driver, target_text):
    base_row_xpath = "/html/body/app-popup/div/div[2]/div[2]/app-ship-details/div/div/table[3]/tbody/tr[NUMBER]/td[1]"
    
    for number in range(2, 42, 2):  # Iterate from 2 to 40 (inclusive), with a step of 2
        row_xpath = base_row_xpath.replace("NUMBER", str(number))
        # Use WebDriverWait to wait for the element to be present and visible
        try:
            element = WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.XPATH, str(row_xpath))))
            if target_text in element.text:
                # If the target_text is found in the row, get the corresponding content xpath
                content_xpath = row_xpath.replace("/td[1]", "/td[2]")
                return content_xpath
            else:
                pass
        except:
            pass
    # Return None if the target_text is not found after 20 attempts
    print("emang gaada bro")
    return None


In [ ]:
def shipdata():
    global LOA, LPP, B, H, T
    global DWT, NT, GT, LT
    global NAME, REGISTER, IMO, vessel_type
    global link
    
    time.sleep(1)
    wait = WebDriverWait(driver, 5) # Maximum wait time of 10 seconds
    
    target_text ="Ship Name:"
    result_xpath = find_xpath_by_text1(driver,target_text)
    NAME = wait.until(EC.visibility_of_element_located((By.XPATH,'{}'.format(result_xpath))))
    
    target_text ="Register Number:"
    result_xpath = find_xpath_by_text1(driver,target_text)
    REGISTER = wait.until(EC.visibility_of_element_located((By.XPATH,'{}'.format(result_xpath))))    
    
    target_text ="IMO Number:"
    result_xpath = find_xpath_by_text1(driver,target_text)
    IMO = wait.until(EC.visibility_of_element_located((By.XPATH,'{}'.format(result_xpath))))   
    
    
    target_text ="Type & service:"
    result_xpath = find_xpath_by_text1(driver,target_text)
    print(result_xpath)

    vessel_type = wait.until(EC.visibility_of_element_located((By.XPATH,'{}'.format(result_xpath))))   
    
    
    # Principal dimension
    target_text ="Overall Length:"
    result_xpath = find_xpath_by_text2(driver,target_text)
    print(result_xpath)
    LOA_text = wait.until(EC.visibility_of_element_located((By.XPATH,'{}'.format("/html/body/app-popup/div/div[2]/div[2]/app-ship-details/div/div/table[3]/tbody/tr[8]/td[2]"))))
    LOA = extract_numeric_value(LOA_text.text)
    
    target_text ="LPP:"
    result_xpath = find_xpath_by_text2(driver,target_text)
    LPP_text = wait.until(EC.visibility_of_element_located((By.XPATH,'{}'.format(result_xpath))))
    LPP = extract_numeric_value(LPP_text.text)
    
    
    target_text ="Breadth:"
    result_xpath = find_xpath_by_text2(driver,target_text)
    B_text = wait.until(EC.visibility_of_element_located((By.XPATH,'{}'.format(result_xpath))))
    B = extract_numeric_value(B_text.text)
    
    try:
        target_text ="Depth:"
        result_xpath = find_xpath_by_text2(driver,target_text)
        H_text = wait.until(EC.visibility_of_element_located((By.XPATH,'{}'.format(result_xpath))))
        H = extract_numeric_value(H_text.text)
    except:
        H = '-'
    
    target_text ="Draught:"
    result_xpath = find_xpath_by_text2(driver,target_text) 
    T_text = wait.until(EC.visibility_of_element_located((By.XPATH,'{}'.format(result_xpath))))
    T = extract_numeric_value(T_text.text)
    
    # Ship tonnage
    try:
        target_text ="Gross Tonnage 69:"
        result_xpath = find_xpath_by_text2(driver,target_text)     
        GT_text = wait.until(EC.visibility_of_element_located((By.XPATH,'{}'.format(result_xpath))))
        GT = extract_numeric_value(GT_text.text)
    except:
        GT = '-'
    
    try:
        target_text ="Net Tonnage 69:"
        result_xpath = find_xpath_by_text2(driver,target_text)  
        NT_text = wait.until(EC.visibility_of_element_located((By.XPATH,'{}'.format(result_xpath))))
        NT = extract_numeric_value(NT_text.text)
    except:
        NT = '-'
        
    try:
        target_text ="Deadweight:"
        result_xpath = find_xpath_by_text2(driver,target_text)
        DWT_text = wait.until(EC.visibility_of_element_located((By.XPATH,'{}'.format(result_xpath))))
        DWT = extract_numeric_value(DWT_text.text)
    except:
        DWT = '-'

    link = 'https://www.vesselfinder.com/vessels/details/{}'.format(IMO.text)

#     # Print statements for extracted data
#     print("Ship Name:", NAME.text)
#     print("Register Number:", REGISTER.text)
#     print("IMO Number:", IMO.text)
#     print("Type & Service:", vessel_type.text)
#     print("Overall Length:", LOA)
#     print("LPP:", LPP)
#     print("Breadth:", B)
#     print("Depth:", H)
#     print("Draught:", T)
#     print("Gross Tonnage:", GT)
#     print("Net Tonnage:", NT)
#     print("Deadweight:", DWT)


In [ ]:
# Define the fieldnames for the CSV file
fieldnames = ['NAME', 'REGISTER', 'IMO', 'TYPE', 'LOA', 'LPP', 'B', 'H', 'T',  'GT', 'NT', 'DWT', 'Link']

In [ ]:
# Set up the webdriver
driver = webdriver.Chrome()

# Counters for ships found and total IMOs
ship_count = 0
total_imo = 0

# Open the input file to get the total number of IMOs
with open(read_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    total_imo = sum(1 for _ in reader)

# Open the CSV file for writing
with open(write_path, mode='w', newline='') as details_file:
    writer = csv.DictWriter(details_file, fieldnames=fieldnames)
    writer.writeheader()

    # Create a progress bar
    progress_bar = tqdm(total=total_imo, desc='Processing IMOs')

    # Open the input file and loop through each vessel name
    with open(read_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None)
        for row in reader:
            IMO = row[0]
            progress_bar.update(1)
            progress_bar.set_postfix({'Ships Found ': ship_count})
            
         
            search(IMO)
            shipdata()
            ship_count += 1
                


            details_dict = {
                'NAME': NAME.text, 'REGISTER': REGISTER.text, 'IMO': IMO.text,
                'TYPE': vessel_type.text, 'LOA': LOA, 'LPP': LPP,
                'B': B, 'H': H, 'T': T, 'GT': GT,
                'NT': NT, 'DWT': DWT, 'Link': link
            }
            writer.writerow(details_dict)


    # Close the progress bar
    progress_bar.close()

# Close the webdriver
driver.quit()

# Display the total number of ships found
print('Total Ships Found:', ship_count)

In [ ]:
# Set up the webdriver
driver = webdriver.Chrome()


IMO = '9659165'

search(IMO)
shipdata()